# Schoology API Exploration

Pull in settings from .env file.

In [4]:
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("SCHOOLOGY_KEY")
API_SECRET = os.getenv("SCHOOLOGY_SECRET")

# Ed-Fi/MSDF users may have this env var set, which causes problems and is unnecessary for the code below
os.environ["REQUESTS_CA_BUNDLE"] = ""

Establish connection to the API

In [5]:
import schoolopy
from prettyprinter import pprint

sc = schoolopy.Schoology(schoolopy.Auth(API_KEY, API_SECRET), verbose = True)

Pull in a list of courses

In [7]:
courses = sc.get_courses()

pprint(courses)

--> calling https://api.schoology.com/v1/courses?limit=20&start=0
[
    schoolopy.models.Course({
        'id': 2941242684,
        'title': 'English I',
        'course_code': 'ENG-1',
        'department': '',
        'description': '',
        'credits': 0,
        'subject_area': 2,
        'grade_level_range_start': 11,
        'grade_level_range_end': 0,
        'synced': 0
    }),
    schoolopy.models.Course({
        'id': 2942191514,
        'title': 'Algebra I',
        'course_code': 'ALG-1',
        'department': '',
        'description': '',
        'credits': 0,
        'subject_area': 3,
        'grade_level_range_start': 11,
        'grade_level_range_end': 0,
        'synced': 0
    })
]


Now pull in a list of all users

In [8]:
users = sc.get_users()
pprint([{u.id, u.name_first, u.name_last} for u in users])

--> calling https://api.schoology.com/v1/users?limit=20&start=0
[
    {100032890, 'Archer', 'Mary'},
    {'Brad', 'Banister', 99785799},
    {'Stephen', 'Caldwell', 100032895},
    {100032898, 'Christian', 'Kelley'},
    {99588912, 'Stephen', 'Fuqua'},
    {99785801, 'Gabrielle'},
    {100032896, 'Hardy', 'Olivia'},
    {100032891, 'Kyle', 'Hughes'},
    {99515016, 'Eric', 'Jansson'},
    {'Larry', 100032893, 'Mahoney'},
    {'Peter', 100032892, 'Nash'},
    {'Phillips', 'Roland', 100032894},
    {100032899, 'Preston', 'Sara'},
    {'Richmond Guzmán', 99785803, 'Luis'},
    {'Micheal', 100032897, 'Turner'}
]


## Extract login and access data from Schoology

_As a school district, I want to know if the student engaged with my my LMS and/or a particular course on a particular school day_

A capture of these data points, or an assessment that these are not available:

1. If a student logged into the LMS on a particular school day
2. A metric of either how long the student was logged in or how many LMSresources (courses, assignments or other system entity) were accessed by a student on a particular day
3. If a student logged into a particular LMS course on a particular school day
4. A metric of either how long the student was logged in to a course or how many LMS resources in that course the student accessed on a particular day

In [10]:
# Find available roles so that we can filter out teachers and admins
roles = sc.get_roles()

ROLE_NAME_STUDENT = "Student"
try:
    role_id_student = next(r.id for r in roles if r.title == ROLE_NAME_STUDENT)
    students = [u for u in users if u.role_id == role_id_student]

    from datetime import datetime, timedelta
    end_time = datetime.now()
    start_time = end_time - timedelta(days=1)

    get_actions = lambda uid : sc.get_user_actions(uid, datetime.timestamp(start_time), datetime.timestamp(end_time))
    
    import json
    actions = [{'First':s.name_first, 'Last':s.name_last, 'Actions':json.dumps(get_actions(s.uid))} for s in students]
    pprint(actions)
except StopIteration:
    print("Role '{role}' does not exist.".format(role=ROLE_NAME_STUDENT))

--> calling https://api.schoology.com/v1/roles?limit=20&start=0
--> calling https://api.schoology.com/v1/analytics/users/100032890?start_time=1598897619.450045&end_time=1598984019.450045&start=0&limit=20?limit=20&start=0
--> calling https://api.schoology.com/v1/analytics/users/100032895?start_time=1598897619.450045&end_time=1598984019.450045&start=0&limit=20?limit=20&start=0
--> calling https://api.schoology.com/v1/analytics/users/100032896?start_time=1598897619.450045&end_time=1598984019.450045&start=0&limit=20?limit=20&start=0
--> calling https://api.schoology.com/v1/analytics/users/100032891?start_time=1598897619.450045&end_time=1598984019.450045&start=0&limit=20?limit=20&start=0
--> calling https://api.schoology.com/v1/analytics/users/100032893?start_time=1598897619.450045&end_time=1598984019.450045&start=0&limit=20?limit=20&start=0
--> calling https://api.schoology.com/v1/analytics/users/100032892?start_time=1598897619.450045&end_time=1598984019.450045&start=0&limit=20?limit=20&st

No analytics available. Have tried many different settings in Schoology, multiple users's key & secret, and cannot find any reason for this. Problems is also born out directly in the site itself: Usage Analytics page returns nothing. Have not been able to reach Schoology for help thus far.